## Notebook Setup  
**Note:** If running on Google Colab, the notebook may automatically restart after reinstalling the correct NumPy version.

In [ ]:
import os
import sys
import subprocess
from pathlib import Path

def is_kaggle():
    return "KAGGLE_URL_BASE" in os.environ or "KAGGLE_KERNEL_RUN_TYPE" in os.environ

def is_colab():
    try:
        import google.colab  # noqa
        return not is_kaggle()
    except ImportError:
        return False

def ensure_numpy_version(required_version="1.26.4"):
    import pkg_resources
    current_version = pkg_resources.get_distribution("numpy").version
    if current_version != required_version:
        print(f"Installing NumPy {required_version} (found {current_version})...")
        subprocess.check_call([
            sys.executable, "-m", "pip", "install",
            f"numpy=={required_version}", "--force-reinstall"
        ])
        print("NumPy version updated. Please restart the runtime.")
        os._exit(0)

def install_project_colab(token, repo_url, repo_dir):
    subprocess.check_call(["git", "clone", f"https://{token}@{repo_url}", repo_dir])
    subprocess.check_call(["pip", "install", repo_dir], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

def install_project_kaggle(token, repo_url):
    subprocess.check_call([
        "pip", "install", f"git+https://{token}@{repo_url}"
    ], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

def configure_environment():
    repo_url = "github.com/Blaku03/PitchAnalyzer_AI.git"
    repo_name = "PitchAnalyzer_AI"

    if is_kaggle():
        print("Running on Kaggle")
        from kaggle_secrets import UserSecretsClient
        token = UserSecretsClient().get_secret("git_token")
        install_project_kaggle(token, repo_url)
        video_path = Path("/kaggle/input/pitchtest/08fd33_4.mp4")
        output_path = f"{video_path.name}_annotated.mp4"

    elif is_colab():
        print("Running on Google Colab")
        from google.colab import userdata
        ensure_numpy_version()
        token = userdata.get("git_token")
        repo_dir = f"/content/{repo_name}"
        install_project_colab(token, repo_url, repo_dir)
        video_path = Path(repo_dir) / "sample_data" / "08fd33_4.mp4"
        output_path = Path(repo_dir) / "output_videos" / f"{video_path.name}_annotated.mp4"
        os.makedirs(output_path.parent, exist_ok=True)

    else:
        print("Running locally")
        ensure_numpy_version()
        get_ipython().run_line_magic("load_ext", "autoreload")
        get_ipython().run_line_magic("autoreload", "2")
        ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
        if ROOT not in sys.path:
            sys.path.insert(0, ROOT)
        video_path = Path("../sample_data/08fd33_4.mp4")
        output_path = f"../output_videos/{video_path.name}_annotated.mp4"
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

    return video_path, output_path

# Call this function in your notebook
video_path, output_path = configure_environment()

## Import models

In [ ]:
from utils.import_models import ModelImporter
field_model = ModelImporter().download_pitch_model()
players_model = ModelImporter().download_player_model()

## Demo of the field mapper

In [ ]:
from utils.notebook_utils import show_image
from utils.video_utils import read_video
from trackers.pitch_tracker import PitchTracker
from annotators.pitch_annotator import PitchAnnotator

frame_generator = read_video(video_path)
first_frame = next(frame_generator)

pitch_tracker = PitchTracker(field_model)
pitch_keypoints, confidence_mask = pitch_tracker.get_pitch_keypoints(first_frame)
pitch_edges = pitch_tracker.get_pitch_edges(first_frame)

show_image(PitchAnnotator.annotate_frame(first_frame.copy(), (pitch_keypoints,pitch_edges)))

In [ ]:
from utils.pitch_utils import draw_pitch, draw_points_on_pitch
from mapping_2d.soccer_field import SoccerPitchConfiguration
import supervision as sv

mapped_vert = pitch_tracker.map_points_2d(first_frame, pitch_keypoints.xy[0])

pitch_img = draw_pitch(SoccerPitchConfiguration())
pitch_img = draw_points_on_pitch(
    config=SoccerPitchConfiguration(),
    xy=mapped_vert.xy[0],
    face_color=sv.Color.BLUE,
    edge_color=sv.Color.BLACK,
    radius=10,
    pitch=pitch_img)

show_image(pitch_img)

## Draw detected players on the 2d plane

In [ ]:
from annotators.game_entities_annotator import GameEntitiesAnnotator
from trackers.game_tracker import GameTracker

frame_generator = read_video(video_path)
tracks_generator =  GameTracker(players_model).get_detections_generator(frame_generator)

first_frame = next(frame_generator)
first_players_tracks = next(tracks_generator)

annotated_sample_image = GameEntitiesAnnotator.annotate_frame(first_frame.copy(), first_players_tracks)
show_image(annotated_sample_image)

In [ ]:
from annotators.pitch_2d_annotator import Pitch2DAnnotator

bottom_boxes = first_players_tracks.players_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
mapped_2d_points = pitch_tracker.map_points_2d(first_frame, bottom_boxes)

show_image(Pitch2DAnnotator().annotate_frame(mapped_2d_points.xy[0], first_players_tracks))

In [ ]:
# Annotate the full video
from utils.video_utils import save_video
import itertools

frame_generator = read_video(video_path)
frame_gen1, frame_gen2 = itertools.tee(frame_generator)

tracks_generator =  GameTracker(players_model).get_detections_generator(frame_gen1)
tracks_gen1, tracks_gen2 = itertools.tee(tracks_generator)

mapped_2d_points_generator = pitch_tracker.map_players_tracks_2d_generator(frame_gen2, tracks_gen1)
annotated_2d_field_generator = Pitch2DAnnotator().annotate_video(mapped_2d_points_generator, tracks_gen2)
save_video(annotated_2d_field_generator, output_path)